In [ ]:
from soldb import main, parse_arguments
import os

# Define Variables
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

username = 'tmind'
param_eval = 'tmind'
mode = 'update'

arguments = ["--username" , username,
              "--eval" , param_eval,
              #"--filter", "C~Biologist" 
                  #"--type", "fuseddeck"  
]

# Now you can use the imported functions
args = parse_arguments(arguments)

# Example usage:

        # F =>  String: Faction                  'Alloyn'              F=Alloyn
        # D =>  String: Deckname                 'Insane'              D~Forge
        # FB => String: Forgeborn               'Ironbeard'            FB=Ironbeard
        # C  => List:   Cardnames               'Digitize'             C~Digitize
        # A  => List:   Forgeborn - Ability     'Army Commander'       A~'Army Commander'
        # K  => Dict:   Composition             'Robot'                K:Robot > 3


In [34]:
from __future__ import print_function
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

from MongoDB.DatabaseManager import DatabaseManager
import GlobalVariables

from pyvis.network import Network
import networkx as nx
import math 
from CardLibrary import Deck
def graphToNet(graph,size=10):
    # Modify the labels of the nodes to include the length of the parents list
    degree_centrality = nx.degree_centrality(graph)
    betweenness_centrality = nx.betweenness_centrality(graph)

    metric = betweenness_centrality

    for node, value in metric.items() :
        decimal = value * size * 1000
        graph.nodes[node]['value'] = decimal
        graph.nodes[node]['label'] = node

    for node, data in graph.nodes(data=True):
        num_parents = len(data.get('parents', []))
        # Add the number of parents to the node's label
        #graph.nodes[node].set('label', "")
        graph.nodes[node]['label'] += f"[{num_parents}]"
        #graph.nodes[node]['value'] = num_parents

    net = Network(notebook=True, directed=True, height="1500px", width="2000px", cdn_resources='in_line')
    net.force_atlas_2based()
    net.from_nx(graph)
    # Display the network
    #net.show_buttons(filter_=['physics'])
    print("Displaying Graph!")
    #display(net.show('graph.html'))
    return net


ValueError: cannot find widget or abbreviation for argument: 'graph'

In [ ]:
from CardLibrary import FusionData, Fusion
import GlobalVariables
from MongoDB.DatabaseManager import DatabaseManager

def create_faction_toggle(faction_names, initial_style='info'):
    faction_toggle = widgets.ToggleButtons(
        options=faction_names,
        description='Faction:',
        disabled=False,
        button_style=initial_style,
        tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
    )

    def update_button_style(change):
        if change['new'] == 'Alloyin':
            faction_toggle.button_style = 'info'
        elif change['new'] == 'Nekrium':
            faction_toggle.button_style = 'warning'
        elif change['new'] == 'Tempys':
            faction_toggle.button_style = 'danger'
        elif change['new'] == 'Uterra':
            faction_toggle.button_style = 'success'

    faction_toggle.observe(update_button_style, 'value')

    return faction_toggle

def button_on_click(button):
    myDeckA = Deck.lookup(dropdown1.value)
    myDeckB = Deck.lookup(dropdown2.value)

    if myDeckA and myDeckB:
        fusionName = f"{myDeckA.name}_{myDeckB.name}"
        fusionCursor = myDB.find('Fusion', {'name' : fusionName})
        if fusionCursor: 
            for fusion in fusionCursor:
                myFusion = Fusion.from_data(fusion)
                display_graph(myFusion)
        else:
            # Create a new fusion based on the decknames
            newFusionData = FusionData(name=fusionName, myDecks=[myDeckA, myDeckB],tags=['forged'] )
            newFusion = Fusion(newFusionData)
            display_graph(newFusion)
                
    else: 
        for deck in [myDeckA , myDeckB] :
            print(deck)
            if deck:
                myGraph = deck.create_graph_children()
                net = graphToNet(myGraph.G)
                display(net.show(f"{deck.name}.html"))

def display_graph(deck):
    myGraph = deck.create_graph_children()
    net = graphToNet(myGraph.G)
    with out:
        out.clear_output() 
        display(net.show(f"{deck.name}.html"))
def update_items(faction_toggle, dropdown):
    deckCursor = myDB.find('Deck', {'faction' : faction_toggle.value})
    deckNames = [deck['name'] for deck in deckCursor]
    dropdown.options = deckNames        

GlobalVariables.username = 'tmind'
myDB = DatabaseManager(GlobalVariables.username)

factionNames = ['Alloyin', 'Nekrium', 'Tempys', 'Uterra']
deckNames = []

out = widgets.Output()

def create_widgets(name) :
    factionToggle = create_faction_toggle(factionNames)
    dropdown = widgets.Dropdown()
    update_items(factionToggle, dropdown)
    factionToggle.observe(lambda change: update_items(factionToggle, dropdown), 'value')
    return factionToggle, dropdown

factionToggle1, dropdown1 = create_widgets('First')       
factionToggle2, dropdown2 = create_widgets('Second')       

button = widgets.Button(description="Show Graph")
button.on_click(button_on_click)


display(factionToggle1, dropdown1, factionToggle2, dropdown2, button)
display(out)




In [ ]:
from CardLibrary import Fusion, FusionData
from MongoDB.DatabaseManager import DatabaseManager
import GlobalVariables
GlobalVariables.username = 'tmind'
myDB = DatabaseManager(GlobalVariables.username)

fusionCursor = myDB.find('Fusion', {'name' : 'Simian - Sailors'})

for fusion in fusionCursor:
    myFusion = Fusion.from_data(fusion)
    myGraph = myFusion.create_graph_children()
    net = graphToNet(myGraph.G)
    display(net.show(f"{myFusion.name}.html"))
    
